In [ ]:
# imports

import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import open3d as o3d

## Functions

In [ ]:
def read_images_to_array(folder_path):

  image_array = []
  # Get a sorted list of filenames
  filenames = sorted(os.listdir(folder_path))
  for filename in filenames:
    if filename.endswith(".jpg") or filename.endswith(".png"):
      img_path = os.path.join(folder_path, filename)
      img = cv2.imread(img_path)

      if img is not None:
        image_array.append(img)

  return image_array

def read_bin_files_to_array(folder_path):
    bin_files = []
    filenames = sorted(os.listdir(folder_path))
    for filename in filenames:
        if filename.endswith('.bin'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                data = np.fromfile(file, dtype=np.float32)
                bin_files.append(data)

    return bin_files


def split_images(image_array): 

    red_region_images = []
    raw_images = [] 

    for image in image_array:
        if image[25,100].sum() == 255*3 :
            red_region_images.append(image)
        else: 
            raw_images.append(image) 
            
    return red_region_images, raw_images

## Apply Functions to Data

In [ ]:
folder_path = './data/Invotive_data'
images = read_images_to_array(folder_path)

masks, images = split_images(images) 

In [ ]:
# sanity check 

rand = random.randint(0, len(images) -1) 
fig, ax = plt.subplots(1,2) 

ax[0].imshow(images[rand])
ax[1].imshow(masks[rand])

print(len(images)) 
print(len(masks))

## Code for bin files 

In [ ]:
# get all bin files from folder
bin_files = read_bin_files_to_array(folder_path) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def read_and_process_point_cloud(file_name):
    # Open the file and read the data
    with open(file_name, 'rb') as fid:
        d = np.fromfile(fid, dtype=np.float32)

    # Reshape the data into a 3-row matrix
    d2 = d.reshape(3, -1)

    # Remove any NaN or Inf values
    valid_mask = np.all(np.isfinite(d2), axis=0)
    d2 = d2[:, valid_mask]

    # Center the data around 0 (optional)
    d2[0, :] -= np.median(d2[0, :])
    d2[1, :] -= np.median(d2[1, :])
    d2[2, :] -= np.median(d2[2, :])

    # Extract x, y, z coordinates
    x = d2[0, :]
    y = d2[1, :]
    z = d2[2, :]

    return x, y, z

def plot_point_cloud(x, y, z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x, y, z, c=z, cmap='viridis', marker='.')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    plt.show()


In [ ]:
rand = random.randint(0, len(bin_files) -1) 
rand_file = bin_files[rand]

x, y, z = read_and_process_point_cloud('data/Invotive_data/L3.3 Parental 2-Control-1-0-38.bin')
point_cloud = [x, y, z]

pcd = o3d.geometry.PointCloud()
pcd.points =  o3d.utility.Vector3dVector(point_cloud[:,:3])

o3d.visualization.draw_geometries([pcd])

In [ ]:
x, y, z = read_and_process_point_cloud('data/Invotive_data/L3.3 Parental 2-Control-1-0-39.bin')
plot_point_cloud(x, y, z)